In [6]:
import numpy as np
import skimage.io
import os
import cv2
import pandas as pd
from scipy import stats
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
import re

In [109]:
Membrane_list=[]
input_dir=r'C:\Users\14194\Documents\Vitroskin_exp\4 days\Day 4 Cell Masks' #Change this to your directory with the images in it

for r, d, f in os.walk(input_dir): #appending to actin list
    for fil in f:
        if (fil.endswith('mask.tif')) & ("beta" in fil):
            Membrane_list.append(os.path.join(input_dir, fil))
        

In [110]:
Membrane_list

['C:\\Users\\14194\\Documents\\Vitroskin_exp\\4 days\\Day 4 Cell Masks\\4_A_0_0_0_NOTEbeta__cell_mask.tif',
 'C:\\Users\\14194\\Documents\\Vitroskin_exp\\4 days\\Day 4 Cell Masks\\4_A_1_0_0_NOTEbeta__cell_mask.tif',
 'C:\\Users\\14194\\Documents\\Vitroskin_exp\\4 days\\Day 4 Cell Masks\\4_A_2_0_0_NOTEbeta__cell_mask.tif',
 'C:\\Users\\14194\\Documents\\Vitroskin_exp\\4 days\\Day 4 Cell Masks\\4_A_4_0_0_NOTEbeta__cell_mask.tif',
 'C:\\Users\\14194\\Documents\\Vitroskin_exp\\4 days\\Day 4 Cell Masks\\4_B_2_0_0_NOTEbeta__cell_mask.tif',
 'C:\\Users\\14194\\Documents\\Vitroskin_exp\\4 days\\Day 4 Cell Masks\\4_B_4_0_0_NOTEbeta__cell_mask.tif',
 'C:\\Users\\14194\\Documents\\Vitroskin_exp\\4 days\\Day 4 Cell Masks\\4_C_0_0_0_NOTEbeta__cell_mask.tif',
 'C:\\Users\\14194\\Documents\\Vitroskin_exp\\4 days\\Day 4 Cell Masks\\4_C_1_0_0_NOTEbeta__cell_mask.tif',
 'C:\\Users\\14194\\Documents\\Vitroskin_exp\\4 days\\Day 4 Cell Masks\\4_C_2_0_0_NOTEbeta__cell_mask.tif',
 'C:\\Users\\14194\\Document

In [11]:
membrane=cv2.imread(Membrane_list[0],-1)

In [64]:
def slice_by_9(img):
    x_dim, y_dim = img.shape
    img1 = img[:int(x_dim/3),:int(y_dim/3)]
    img2 = img[int(x_dim/3):int(2*x_dim/3),:int(y_dim/3)]
    img3 = img[int(2*x_dim/3):,:int(y_dim/3)]
    img4 = img[:int(x_dim/3),int(y_dim/3):int(2*y_dim/3)]
    img5 = img[int(x_dim/3):int(2*x_dim/3),int(y_dim/3):int(2*y_dim/3)]
    img6 = img[int(2*x_dim/3):,int(y_dim/3):int(2*y_dim/3)]
    img7 = img[:int(x_dim/3),int(2*y_dim/3):]
    img8 = img[int(x_dim/3):int(2*x_dim/3),int(2*y_dim/3):]
    img9 = img[int(2*x_dim/3):,int(2*y_dim/3):]
    return [img1, img2, img3, img4, img5, img6, img7, img8, img9]

In [111]:
Membrane_File=[]
image_num = []
Pixel_Coordinate=[]
Cell_Area=[]
eccentricity = []

for i in range(len(Membrane_list)):
    membrane=cv2.imread(Membrane_list[i],-1)
    image_sliced = slice_by_9(membrane)
    print(Membrane_list[i])
    for j in range(len(image_sliced)):
        slice = image_sliced[j]
        for k in np.unique(slice)[1:]:        
            regions = regionprops((slice == k).astype(int))
            eccentricity.append(regions[0].eccentricity)
            Membrane_File.append(os.path.basename(Membrane_list[i]))#cell name of mask
            Pixel_Coordinate.append(k) #pixel coordinate of a cell
            Cell_Area.append(np.count_nonzero(membrane == k)) #same as above but for cell
            image_num.append(j)

C:\Users\14194\Documents\Vitroskin_exp\4 days\Day 4 Cell Masks\4_A_0_0_0_NOTEbeta__cell_mask.tif
C:\Users\14194\Documents\Vitroskin_exp\4 days\Day 4 Cell Masks\4_A_1_0_0_NOTEbeta__cell_mask.tif
C:\Users\14194\Documents\Vitroskin_exp\4 days\Day 4 Cell Masks\4_A_2_0_0_NOTEbeta__cell_mask.tif
C:\Users\14194\Documents\Vitroskin_exp\4 days\Day 4 Cell Masks\4_A_4_0_0_NOTEbeta__cell_mask.tif
C:\Users\14194\Documents\Vitroskin_exp\4 days\Day 4 Cell Masks\4_B_2_0_0_NOTEbeta__cell_mask.tif
C:\Users\14194\Documents\Vitroskin_exp\4 days\Day 4 Cell Masks\4_B_4_0_0_NOTEbeta__cell_mask.tif
C:\Users\14194\Documents\Vitroskin_exp\4 days\Day 4 Cell Masks\4_C_0_0_0_NOTEbeta__cell_mask.tif
C:\Users\14194\Documents\Vitroskin_exp\4 days\Day 4 Cell Masks\4_C_1_0_0_NOTEbeta__cell_mask.tif
C:\Users\14194\Documents\Vitroskin_exp\4 days\Day 4 Cell Masks\4_C_2_0_0_NOTEbeta__cell_mask.tif
C:\Users\14194\Documents\Vitroskin_exp\4 days\Day 4 Cell Masks\4_C_4_0_0_NOTEbeta__cell_mask.tif


In [112]:
Master_Sheet=pd.DataFrame() # creating dataframe
Master_Sheet['Membrane_File']=Membrane_File
Master_Sheet['Pixel_Coordinate']=Pixel_Coordinate
Master_Sheet['image_num'] = image_num
Master_Sheet['Cell_Area (pixels)']=Cell_Area
Master_Sheet["Eccentricity"] = eccentricity

In [114]:
Master_Sheet.to_csv(r"C:\Users\14194\Documents\Vitroskin_exp\4 days\MS_4_day.csv")

In [113]:
Master_Sheet["file, image_num"] = Master_Sheet.apply(lambda x: str([x["Membrane_File"], x["image_num"]]),axis = 1)

In [115]:
MS_25days = pd.read_csv(r"C:\Users\14194\Documents\Vitroskin_exp\25 days\MS_25_day.csv")
MS_11days = pd.read_csv(r"C:\Users\14194\Documents\Vitroskin_exp\11 days\MS_11_day.csv")
MS_4days = pd.read_csv(r"C:\Users\14194\Documents\Vitroskin_exp\4 days\MS_4_day.csv")
MS_1days = pd.read_csv(r"C:\Users\14194\Documents\Vitroskin_exp\1 day\MS_1_day.csv")

In [126]:
image_index = np.concatenate([pd.unique(MS_25days["file, image_num"]), 
                         pd.unique(MS_11days["file, image_num"]), 
                         pd.unique(MS_4days["file, image_num"]), 
                         pd.unique(MS_1days["file, image_num"])])
image_index

array(["['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 0]",
       "['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 1]",
       "['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 2]",
       "['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 3]",
       "['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 4]",
       "['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 5]",
       "['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 6]",
       "['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 7]",
       "['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 8]",
       "['25_A_0_0_1_NOTEbeta__cell_mask.tif', 0]",
       "['25_A_0_0_1_NOTEbeta__cell_mask.tif', 1]",
       "['25_A_0_0_1_NOTEbeta__cell_mask.tif', 2]",
       "['25_A_0_0_1_NOTEbeta__cell_mask.tif', 3]",
       "['25_A_0_0_1_NOTEbeta__cell_mask.tif', 4]",
       "['25_A_0_0_1_NOTEbeta__cell_mask.tif', 5]",
       "['25_A_0_0_1_NOTEbeta__cell_mask.tif', 6]",
       "['25_A_0_0_1_NOTEbeta__cell_mask.tif', 7]",
       "['25_A_0_0_1_NOTEbeta__cell_mask.tif', 8]",
       "['25_A_1_0_0_NOTEbet

In [127]:
file_list_beta = ['1_A_0_0_0_NOTEbeta__cell_mask.tif',
                  '1_A_1_0_0_NOTEbeta__cell_mask.tif',
                  '1_A_2_0_0_NOTEbeta__cell_mask.tif',
                  '1_A_4_0_0_NOTEbeta__cell_mask.tif',
                  '1_B_0_0_0_NOTEbeta__cell_mask.tif',
                  '1_B_1_0_0_NOTEbetacrop__cell_mask.tif',
                  '1_B_2_0_0_NOTEbeta__cell_mask.tif',
                  '1_B_4_0_0_NOTEbeta__cell_mask.tif',
                  '1_C_0_0_0_NOTEbetacrop__cell_mask.tif',
       '4_A_0_0_0_NOTEbeta__cell_mask.tif',
       '4_A_1_0_0_NOTEbeta__cell_mask.tif',
       '4_A_2_0_0_NOTEbeta__cell_mask.tif',
       '4_A_4_0_0_NOTEbeta__cell_mask.tif',
       '4_B_2_0_0_NOTEbeta__cell_mask.tif',
       '4_B_4_0_0_NOTEbeta__cell_mask.tif',
       '4_C_0_0_0_NOTEbeta__cell_mask.tif',
       '4_C_1_0_0_NOTEbeta__cell_mask.tif',
       '4_C_2_0_0_NOTEbeta__cell_mask.tif',
       '4_C_4_0_0_NOTEbeta__cell_mask.tif',
       '11_A_0_0_0_NOTEbeta__cell_mask.tif',
       '11_A_1_0_0_NOTEbeta__cell_mask.tif',
       '11_A_2_0_0_NOTEbeta__cell_mask.tif',
       '11_A_4_0_0_NOTEbeta__cell_mask.tif',
       '11_B_0_0_0_NOTEbeta__cell_mask.tif',
       '11_B_1_0_0_NOTEbeta__cell_mask.tif',
       '11_B_2_0_0_NOTEbeta__cell_mask.tif',
       '11_B_4_0_0_NOTEbeta__cell_mask.tif',
       '11_C_0_0_0_NOTEbeta__cell_mask.tif',
       '11_C_1_0_0_NOTEbeta__cell_mask.tif',
       '11_C_2_0_0_NOTEbeta__cell_mask.tif',
       '25_A_0_0_0_NOTEbetacrop__cell_mask.tif',
       '25_A_0_0_1_NOTEbeta__cell_mask.tif',
       '25_A_1_0_0_NOTEbeta__cell_mask.tif',
       '25_A_1_0_1_NOTEbeta__cell_mask.tif',
       '25_A_2_0_0_NOTEbeta__cell_mask.tif',
       '25_A_4_0_0_NOTEbeta__cell_mask.tif',
       '25_A_4_0_1_NOTEbeta__cell_mask.tif',
       '25_B_0_0_0_NOTEbeta__cell_mask.tif',
       '25_B_1_0_0_NOTEbeta__cell_mask.tif',
       '25_B_2_0_1_NOTEbeta__cell_mask.tif',
       '25_B_4_0_0_NOTEbeta__cell_mask.tif',
       '25_C_0_0_1_NOTEbeta__cell_mask.tif',
       '25_C_1_0_0_NOTEbeta__cell_mask.tif',
       '25_C_2_0_0_NOTEbeta__cell_mask.tif',
       '25_C_4_0_0_NOTEbeta__cell_mask.tif',
       '25_D_0_0_0_NOTEbeta__cell_mask.tif',
       '25_D_1_0_0_NOTEbeta__cell_mask.tif',
       '25_D_2_0_0_NOTEbeta__cell_mask.tif',
       '25_D_2_0_1_NOTEbeta__cell_mask.tif',
       '25_D_4_0_0_NOTEbeta__cell_mask.tif']

In [128]:
class_list = [
                "",
                "Bacilli",
                "Bacilli",
                "Bacilli",
                "Bacilli",
                "Bacilli",
                "Bacilli",
                "Bacilli",
                "Bacilli",
                "Alphaproteobacteria",
                "Bacilli",
                "Gammaproteobacteria",
                "Bacilli",
                "Gammaproteobacteria",
                "Actinobacteria",
                "Alphaproteobacteria",
                "Alphaproteobacteria",
                "Actinobacteria",
                "Bacilli",
                "Actinobacteria",
                "Actinobacteria",
                "Alphaproteobacteria",
                "Actinobacteria",
                "Actinobacteria",
                "Actinobacteria",
                "Bacilli",
                "Alphaproteobacteria",
                "Actinobacteria",
                "Actinobacteria",
                "Bacilli",
                "Actinobacteria",
                "Actinobacteria",
                "Actinobacteria",
                "Actinobacteria",
                "",
                "Actinobacteria",
                "Actinobacteria",
                "Actinobacteria",
                "Actinobacteria",
                "Actinobacteria",
                "Alphaproteobacteria",
                "Actinobacteria",
                "Actinobacteria",
                "Actinobacteria",
                "Actinobacteria",
                "Alphaproteobacteria",
                "Alphaproteobacteria",
                "Alphaproteobacteria",
                "Alphaproteobacteria",
                ""
             ]

In [129]:
shannon_list = [
0,
0.95964377,
0.316730651,
0.224232243,
0.397380735,
0.569870758,
0.718275847,
0.847194281,
0.920380107,
0.837513433,
1.04708066,
1.088708035,
0.449720537,
1.363090426,
1.116414568,
1.44174157,
0.557031823,
1.050723489,
1.061441262,
0.807828021,
0.864918072,
1.051878533,
0.518650296,
0.493147643,
0.66653307,
1.232381531,
0.500322651,
0.995515816,
0.702259758,
0.559632186,
0.61217081,
0.61217081,
0.074071185,
0.074071185,
0,
0.584403582,
0.584403582,
0.191884077,
0.132843321,
0.695358217,
1.031308317,
0.523505501,
0.201574998,
0.975963524,
0.793394966,
0.778593029,
0.843078906,
0.691246545,
0.691246545,
0
]

In [143]:
percent_tax_list = [
    0,    
    57.85104086,
    94.26060622,
    98.42114268,
    90.92676342,
    86.15400378,
    62.92042482,
    79.15378338,
    59.73476582,
    78.91632284,
    53.56457679,
    52.33919999,
    92.10397436,
    40.03628813,
    48.34009828,
    50.42428668,
    89.73183969,
    66.78212463,
    59.30607855,
    71.54337722,
    72.29849128,
    68.91789739,
    87.42278618,
    92.80752441,
    89.88336139,
    41.53283641,
    90.35748491,
    49.65476478,
    80.24006501,
    89.67735136,
    84.19080909,
    84.19080909,
    99.09351582,
    99.09351582,
    0,
    79.47189733,
    79.47189733,
    97.06312792,
    97.93875153,
    78.87678084,
    49.94110963,
    86.0482364,
    96.64297238,
    66.2073699,
    76.96499074,
    73.39105656,
    60.86759067,
    82.91198025,
    82.91198025,
    0
]

In [136]:
image_index[0].split(",")[0][2:-1]

'25_A_0_0_0_NOTEbetacrop__cell_mask.tif'

In [140]:
df_eccen_arrays = pd.DataFrame(index = image_index, columns = ["Membrane_File", "Array", "class", "tax_percent", "shannon","count"])
for i in image_index:
    if i[2:4] == "1_":
        data1 = np.array([MS_1days[(MS_1days["file, image_num"] == i)]["Cell_Area (pixels)"],
                          MS_1days[(MS_1days["file, image_num"] == i)]["Eccentricity"]])
    if i[2:4] == "4_":
        data1 = np.array([MS_4days[(MS_4days["file, image_num"] == i)]["Cell_Area (pixels)"],
                          MS_4days[(MS_4days["file, image_num"] == i)]["Eccentricity"]])
    if i[2:4] == "11":
        data1 = np.array([MS_11days[(MS_11days["file, image_num"] == i)]["Cell_Area (pixels)"],
                          MS_11days[(MS_11days["file, image_num"] == i)]["Eccentricity"]])
    if i[2:4] == "25":
        data1 = np.array([MS_25days[(MS_25days["file, image_num"] == i)]["Cell_Area (pixels)"],
                          MS_25days[(MS_25days["file, image_num"] == i)]["Eccentricity"]])
    X, Y = np.mgrid[0:200:200j, 0:1:200j]
    positions = np.vstack([X.ravel(), Y.ravel()])
    kde1 = stats.gaussian_kde(data1)
    print(i)
    temp = np.reshape(kde1(positions).T,X.shape)
    df_eccen_arrays["Array"][i] = (temp)/(sum(map(sum,temp)))
    df_eccen_arrays["Membrane_File"][i] = i.split(",")[0][2:-1]
    df_eccen_arrays["count"][i] = len(data1[0])
    print(len(data1[0]))

['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 0]
1037
['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 1]
1146
['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 2]
1084
['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 3]
984
['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 4]
998
['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 5]
969
['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 6]
938
['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 7]
915
['25_A_0_0_0_NOTEbetacrop__cell_mask.tif', 8]
876
['25_A_0_0_1_NOTEbeta__cell_mask.tif', 0]
1815
['25_A_0_0_1_NOTEbeta__cell_mask.tif', 1]
1221
['25_A_0_0_1_NOTEbeta__cell_mask.tif', 2]
1265
['25_A_0_0_1_NOTEbeta__cell_mask.tif', 3]
1745
['25_A_0_0_1_NOTEbeta__cell_mask.tif', 4]
1514
['25_A_0_0_1_NOTEbeta__cell_mask.tif', 5]
1633
['25_A_0_0_1_NOTEbeta__cell_mask.tif', 6]
2057
['25_A_0_0_1_NOTEbeta__cell_mask.tif', 7]
1597
['25_A_0_0_1_NOTEbeta__cell_mask.tif', 8]
1736
['25_A_1_0_0_NOTEbeta__cell_mask.tif', 0]
2550
['25_A_1_0_0_NOTEbeta__cell_mask.tif', 1]
2335
['25_A_1_0_0_NOTEbeta__cell_ma

In [141]:
df_dict = pd.DataFrame(columns = ["Membrane_File","class", "tax_percent", "shannon"])


In [145]:
df_dict

,Membrane_File,class,tax_percent,shannon
0,1_A_0_0_0_NOTEbeta__cell_mask.tif,,0.000000,0.000000
1,1_A_1_0_0_NOTEbeta__cell_mask.tif,Bacilli,57.851041,0.959644
2,1_A_2_0_0_NOTEbeta__cell_mask.tif,Bacilli,94.260606,0.316731
3,1_A_4_0_0_NOTEbeta__cell_mask.tif,Bacilli,98.421143,0.224232
4,1_B_0_0_0_NOTEbeta__cell_mask.tif,Bacilli,90.926763,0.397381
5,1_B_1_0_0_NOTEbetacrop__cell_mask.tif,Bacilli,86.154004,0.569871
6,1_B_2_0_0_NOTEbeta__cell_mask.tif,Bacilli,62.920425,0.718276
7,1_B_4_0_0_NOTEbeta__cell_mask.tif,Bacilli,79.153783,0.847194
8,1_C_0_0_0_NOTEbetacrop__cell_mask.tif,Bacilli,59.734766,0.920380
9,4_A_0_0_0_NOTEbeta__cell_mask.tif,Alphaproteobacteria,78.916323,0.837513


In [144]:
df_dict["Membrane_File"] = file_list_beta
df_dict["class"] = class_list
df_dict["tax_percent"] = percent_tax_list
df_dict["shannon"] = shannon_list


In [148]:
df_joined = pd.merge(df_eccen_arrays, df_dict, how="left", on = ["Membrane_File"])

In [154]:

df_joined.pop("shannon_x")

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
445    NaN
446    NaN
447    NaN
448    NaN
449    NaN
Name: shannon_x, Length: 450, dtype: object

In [155]:
df_joined

,Membrane_File,Array,count,class_y,tax_percent_y,shannon_y
0,25_A_0_0_0_NOTEbetacrop__cell_mask.tif,"[[5.081872676893776e-08, 4.8460704058174135e-0...",1037,Actinobacteria,84.190809,0.612171
1,25_A_0_0_0_NOTEbetacrop__cell_mask.tif,"[[2.393376562270774e-10, 2.2821606319656274e-1...",1146,Actinobacteria,84.190809,0.612171
2,25_A_0_0_0_NOTEbetacrop__cell_mask.tif,"[[2.54735757976638e-08, 2.365368518778207e-08,...",1084,Actinobacteria,84.190809,0.612171
3,25_A_0_0_0_NOTEbetacrop__cell_mask.tif,"[[1.4798096117908975e-08, 1.3649336389485242e-...",984,Actinobacteria,84.190809,0.612171
4,25_A_0_0_0_NOTEbetacrop__cell_mask.tif,"[[1.923769215792163e-08, 1.783248310583817e-08...",998,Actinobacteria,84.190809,0.612171
...,...,...,...,...,...,...
445,1_C_0_0_0_NOTEbetacrop__cell_mask.tif,"[[1.6660200937233115e-07, 1.5330284203104835e-...",1462,Bacilli,59.734766,0.920380
446,1_C_0_0_0_NOTEbetacrop__cell_mask.tif,"[[1.714680357669114e-07, 1.5447716442959433e-0...",2410,Bacilli,59.734766,0.920380
447,1_C_0_0_0_NOTEbetacrop__cell_mask.tif,"[[1.600410274475345e-06, 1.536875237196635e-06...",436,Bacilli,59.734766,0.920380
448,1_C_0_0_0_NOTEbetacrop__cell_mask.tif,"[[1.4973179505061335e-07, 1.4227925409700885e-...",677,Bacilli,59.734766,0.920380


In [156]:
dataset = df_joined[(df_joined["count"] >= 100) & (df_joined["tax_percent_y"]>0)]

In [157]:
dataset

,Membrane_File,Array,count,class_y,tax_percent_y,shannon_y
0,25_A_0_0_0_NOTEbetacrop__cell_mask.tif,"[[5.081872676893776e-08, 4.8460704058174135e-0...",1037,Actinobacteria,84.190809,0.612171
1,25_A_0_0_0_NOTEbetacrop__cell_mask.tif,"[[2.393376562270774e-10, 2.2821606319656274e-1...",1146,Actinobacteria,84.190809,0.612171
2,25_A_0_0_0_NOTEbetacrop__cell_mask.tif,"[[2.54735757976638e-08, 2.365368518778207e-08,...",1084,Actinobacteria,84.190809,0.612171
3,25_A_0_0_0_NOTEbetacrop__cell_mask.tif,"[[1.4798096117908975e-08, 1.3649336389485242e-...",984,Actinobacteria,84.190809,0.612171
4,25_A_0_0_0_NOTEbetacrop__cell_mask.tif,"[[1.923769215792163e-08, 1.783248310583817e-08...",998,Actinobacteria,84.190809,0.612171
...,...,...,...,...,...,...
445,1_C_0_0_0_NOTEbetacrop__cell_mask.tif,"[[1.6660200937233115e-07, 1.5330284203104835e-...",1462,Bacilli,59.734766,0.920380
446,1_C_0_0_0_NOTEbetacrop__cell_mask.tif,"[[1.714680357669114e-07, 1.5447716442959433e-0...",2410,Bacilli,59.734766,0.920380
447,1_C_0_0_0_NOTEbetacrop__cell_mask.tif,"[[1.600410274475345e-06, 1.536875237196635e-06...",436,Bacilli,59.734766,0.920380
448,1_C_0_0_0_NOTEbetacrop__cell_mask.tif,"[[1.4973179505061335e-07, 1.4227925409700885e-...",677,Bacilli,59.734766,0.920380


In [158]:
import pickle

with open('preprocessed_data.pkl', 'wb') as file: 
      
    # A new file will be created 
    pickle.dump(dataset, file) 

In [159]:
file = open("preprocessed_data.pkl", 'rb')
df = pickle.load(file)

In [162]:
df["Array"][0].shape

(200, 200)